In [3]:
import cv2
import numpy as np
import mediapipe as mp

In [4]:
mp_face_mesh = mp.solutions.face_mesh

face_mesh = mp_face_mesh.FaceMesh(
    max_num_faces=1,
    static_image_mode=False,
    min_detection_confidence=0.8,
    min_tracking_confidence=0.8
)

In [5]:
def get_landmarks(image):
    landmarks = {}
    h,w = image.shape[:2]
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    face_mesh_result = face_mesh.process(image)
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    
    if face_mesh_result.multi_face_landmarks:
        for i, landmark in enumerate(face_mesh_result.multi_face_landmarks[0].landmark): 
            x = landmark.x
            y = landmark.y
            relative_x = int(x * w)
            relative_y = int(y * h)
            landmarks[i+1] = (relative_x, relative_y)
    return landmarks


In [22]:
def get_coordinates(landmarks):
    coordinates = {
        "eye_left": (landmarks[30], (landmarks[158][0], landmarks[145][1])),
        "eye_right": (landmarks[287], (landmarks[260][0], landmarks[381][1])),
        "shade": (landmarks[71], (landmarks[294][0], landmarks[119][1])),
    }
    return coordinates

In [7]:
def add_effect(image, effect, icon_path, cordinates):
    item = cv2.imread(icon_path)
    pt1, pt2 = cordinates[effect]
    x, y, x_w, y_h = pt1[0], pt1[1], pt2[0], pt2[1]
    cropped = image[y:y_h, x:x_w, :]
    h, w, _ = cropped.shape
    item = cv2.resize(item, (w, h))
    blend = cv2.addWeighted(cropped, 0, item, 1.0, 0)
    
    return blend, x, y, x_w, y_h

In [8]:
def remove_whitespace(image, blend, x, y, threshold=225):
    for i in range(blend.shape[0]):
        for j in range(blend.shape[1]):
            for k in range(3):
                if blend[i][j][k] > threshold:
                    blend[i][j][k] = image[i + y][j + x][k]

In [25]:
def draw_iris_effects(image, coordinates):
    icon_path = "effects\\eye.png"
    for effect in ['eye_left', 'eye_right']:
        blend, x, y, x_w, y_h = add_effect(image, effect, icon_path, coordinates)
        remove_whitespace(
            image=image,
            blend=blend,
            x=x,
            y=y,
            # threshold=200
        )
        image[y:y_h, x:x_w, :] = blend

In [26]:
cam = cv2.VideoCapture(0)

while True:
    ret, frame = cam.read()
    
    if ret:
        height, width, _ = frame.shape
        image = cv2.resize(frame, (width//2, height//2))[:,::-1]
        landmarks = get_landmarks(image=image)
        print(landmarks[30])
        try:
            coordinates = get_coordinates(landmarks=landmarks)
            draw_iris_effects(image, coordinates)
        except KeyError as k:
            print(k)

        cv2.imshow("Live face effects", image)
        if cv2.waitKey(5) & 0xFF == 27:
            cv2.destroyAllWindows() 
            cam.release()
            break

(100, 106)
(99, 106)
(100, 106)
(99, 105)
(99, 105)
(99, 105)
(99, 104)
(98, 104)
(98, 104)
(98, 105)
(98, 105)
(98, 105)
(98, 105)
(98, 105)
(98, 105)
(98, 105)
(98, 105)
(98, 105)
(98, 105)
(98, 106)
(98, 106)
(99, 107)
(99, 107)
(99, 107)
(99, 107)
(98, 108)
(98, 108)
(99, 108)
(99, 108)
(98, 107)
(99, 107)
(99, 108)
(99, 108)
(99, 107)
(99, 107)
(99, 108)
(99, 108)
(98, 108)
(98, 108)
(98, 108)
(98, 108)
(97, 109)
(97, 109)
(97, 109)
(98, 109)
(98, 109)
(98, 109)
(98, 109)
(98, 109)
(97, 110)
(97, 110)
(97, 110)
